In [ ]:
from diffusers import StableDiffusionPipeline
import torch

pipe = StableDiffusionPipeline.from_pretrained(
    "runwayml/stable-diffusion-v1-5",
    torch_dtype=torch.float16
)
pipe = pipe.to("cuda")

# pipe.load_lora_weights("latent-consistency/lcm-lora-sdv1-5")

In [ ]:
import torch
from diffusers import LCMScheduler, AutoPipelineForText2Image

model_id = "Lykon/dreamshaper-7"
adapter_id = "latent-consistency/lcm-lora-sdv1-5"

pipe = AutoPipelineForText2Image.from_pretrained(model_id, torch_dtype=torch.float16, variant="fp16")
pipe.scheduler = LCMScheduler.from_config(pipe.scheduler.config)
pipe.to("cuda")

# load and fuse lcm lora
pipe.load_lora_weights(adapter_id)
pipe.fuse_lora()


prompt = "Self-portrait oil painting, a beautiful cyborg with golden hair, 8k"

# disable guidance_scale by passing 0
image = pipe(prompt=prompt, num_inference_steps=4, guidance_scale=0).images[0]


In [ ]:
import matplotlib.pyplot as plt

# 이미지 생성
prompt = "A cute little girl wearing a red hooded cape, big bright eyes, soft smile, fairy tale style, standing in a magical forest, whimsical and dreamy atmosphere, highly detailed, vibrant colors, Studio Ghibli style"
image = pipe(prompt).images[0]

# matplotlib으로 표시
plt.figure(figsize=(10, 10))
plt.imshow(image)
plt.axis('off')
plt.title("Red Hood")
plt.show()

In [ ]:
import torch
import gc

# 기존 파이프라인 삭제
del pipe
torch.cuda.empty_cache()
gc.collect()

# 메모리 확인
print(f"GPU 메모리: {torch.cuda.memory_allocated()/1024**3:.2f} GB")